In [ ]:
## merge table to automatically find the key left join columns and match items 

In [1]:
# Standard Library
import os
import time
import re
import json
import gzip
import shutil
import tarfile
import subprocess
from pathlib import Path
from typing import TypedDict, List, Dict, Any
from urllib.parse import urljoin, urlparse, parse_qs, unquote

# Third-Party Libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import display, Markdown
from geofetch import Geofetcher

# Bioinformatics
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.default_inference import DefaultInference
import gseapy as gp
from geofetch import Geofetcher
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import scanpy as sc
from sanbomics.plots import volcano

# LangChain/LangGraph
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda

In [3]:
API_KEY = ''
os.environ['OPENAI_API_KEY']=API_KEY
client = OpenAI()

In [83]:
a=pd.read_csv("./data_folder/cancer_cancers_master.csv")
b=pd.read_csv("outcome_table.csv")

In [8]:
import os
import json
import pandas as pd
import re
from typing import Dict
from openai import OpenAI

# === TCGA abbreviation map ===
TCGA_MAP = {
    "ACC": "Adrenocortical carcinoma",
    "BLCA": "Bladder Urothelial Carcinoma",
    "BRCA": "Breast invasive carcinoma",
    "CESC": "Cervical squamous cell carcinoma and endocervical adenocarcinoma",
    "CHOL": "Cholangiocarcinoma",
    "COAD": "Colon adenocarcinoma",
    "DLBC": "Lymphoid Neoplasm Diffuse Large B-cell Lymphoma",
    "ESCA": "Esophageal carcinoma",
    "GBM": "Glioblastoma multiforme",
    "HNSC": "Head and Neck squamous cell carcinoma",
    "KICH": "Kidney Chromophobe",
    "KIRC": "Kidney renal clear cell carcinoma",
    "KIRP": "Kidney renal papillary cell carcinoma",
    "LAML": "Acute Myeloid Leukemia",
    "LGG": "Brain Lower Grade Glioma",
    "LIHC": "Liver hepatocellular carcinoma",
    "LUAD": "Lung adenocarcinoma",
    "LUSC": "Lung squamous cell carcinoma",
    "MESO": "Mesothelioma",
    "OV": "Ovarian serous cystadenocarcinoma",
    "PAAD": "Pancreatic adenocarcinoma",
    "PCPG": "Pheochromocytoma and Paraganglioma",
    "PRAD": "Prostate adenocarcinoma",
    "READ": "Rectum adenocarcinoma",
    "SARC": "Sarcoma",
    "SKCM": "Skin Cutaneous Melanoma",
    "STAD": "Stomach adenocarcinoma",
    "TGCT": "Testicular Germ Cell Tumors",
    "THCA": "Thyroid carcinoma",
    "THYM": "Thymoma",
    "UCEC": "Uterine Corpus Endometrial Carcinoma",
    "UCS": "Uterine Carcinosarcoma",
    "UVM": "Uveal Melanoma"
}

# === Normalize function ===
# === Normalize function with keyword matching ===
def normalize_cancer_name(name: str) -> str:
    if not isinstance(name, str):
        return ""
    name = name.strip().lower()

    # Exact match
    for abbr, full in TCGA_MAP.items():
        if abbr.lower() in name or full.lower() in name:
            return abbr

    # Fuzzy keyword matching
    keyword_map = {
    
        "breast": "BRCA",
        "carcinoid": "NEC",  # Not in TCGA; placeholder if needed
        "cervical": "CESC",
        "colorectal": "COAD",  # could also be READ
        "endometrial": "UCEC",
        "glioma": "LGG",
        "head and neck": "HNSC",
        "liver": "LIHC",
        "lung": "LUAD",  # or LUSC
        "lymphoma": "DLBC",
        "melanoma": "SKCM",
        "ovarian": "OV",
        "pancreatic": "PAAD",
        "prostate": "PRAD",
        "renal": "KIRC",  # or KIRP or KICH
        "skin": "SKCM",
        "stomach": "STAD",
        "thyroid": "THCA",
        "testicular": "TGCT",
        "bladder": "BLCA",
        "esophageal": "ESCA",
        "leukemia": "LAML",
        "thymoma": "THYM",
        "sarcoma": "SARC",
        "mesothelioma": "MESO",
        "uterine": "UCEC",
        "uveal": "UVM"
    
    }

    for keyword, abbr in keyword_map.items():
        if keyword in name:
            return abbr

    return ""

# === Prompt formatting ===
def generate_key_column_prompt(master_cols, other_cols):
    return f"""
You are given two tables with the following column names:
Master table: {master_cols}
Other table: {other_cols}

Identify which column in each table most likely refers to the cancer type or cancer abbreviation.
Return your answer as a JSON with keys 'master_key' and 'other_key'.
"""

def generate_value_mapping_prompt(master_values, other_values):
    return f"""
Your task is to match cancer type names from two datasets.
Normalize names from the second list to match TCGA abbreviations from the first.
Use this TCGA list:
{json.dumps(TCGA_MAP, indent=2)}

Master list:
{master_values}

Other list:
{other_values}

Return a JSON dictionary where keys are values from the other list and values are matched TCGA codes.
"""

# === JSON parse with recovery ===
def safe_parse_json(content: str) -> Dict:
    try:
        return json.loads(content.strip())
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", content, re.DOTALL)
        if match:
            return json.loads(match.group())
        return {}

# === Step 1: Normalize both tables ===
def attach_normalized_keys(master_df, other_df, client: OpenAI):
    col_prompt = generate_key_column_prompt(master_df.columns.tolist(), other_df.columns.tolist())
    col_response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "user", "content": col_prompt}]
    )
    col_keys = safe_parse_json(col_response.choices[0].message.content)
    master_key = col_keys.get("master_key")
    other_key = col_keys.get("other_key")

    if master_key not in master_df.columns or other_key not in other_df.columns:
        raise ValueError("Key columns not found")

    master_df['__norm_key__'] = master_df[master_key].apply(normalize_cancer_name)
    master_values = list(master_df['__norm_key__'].dropna().unique())
    other_values = list(other_df[other_key].dropna().unique())

    prompt = generate_value_mapping_prompt(master_values, other_values)
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )

    value_map = safe_parse_json(response.choices[0].message.content)

    if not isinstance(value_map, dict):
        raise ValueError("LLM did not return a valid mapping")

    other_df['__norm_key__'] = other_df[other_key].map(value_map).fillna(
        other_df[other_key].apply(normalize_cancer_name)
    )

    print("🧪 Master table head with __norm_key__:")
    print(master_df.head(3))
    print("📄 Other table head with __norm_key__:")
    print(other_df.head(3))

    return master_df, other_df

# === Step 2: Full semantic merge ===
def merge_tables_semantically(master_path: str, folder: str, client: OpenAI) -> pd.DataFrame:
    master_df = pd.read_csv(master_path)

    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        if not filename.endswith(".csv") or os.path.abspath(path) == os.path.abspath(master_path):
            continue

        print(f"\n\U0001f4c4 Processing {filename}")
        other_df = pd.read_csv(path)

        try:
            master_df, other_df = attach_normalized_keys(master_df, other_df, client)
            result = pd.merge(master_df, other_df, on="__norm_key__", how="left", 
                              suffixes=("", f"_{filename.replace('.csv', '')}"))
            master_df = result
            print("✅ Merge done. Preview:")
            print(master_df.head(5))
        except Exception as e:
            print(f"❌ Skipping {filename}: {e}")

    return master_df

In [9]:
master_df = pd.read_csv("./data_folder/cancer_cancers_master.csv")
other_df = pd.read_csv("./data_folder/outcome_table.csv")  # change this as needed

master_normed, other_normed = attach_normalized_keys(master_df, other_df, client)
print("\n✅ Test complete. Normalized DataFrames:")
print(master_normed[['Cancer', '__norm_key__']].head())
print(other_normed[['cancer', '__norm_key__']].head())
result = pd.merge(master_normed, other_normed, on="__norm_key__")

🧪 Master table head with __norm_key__:
            Cancer                                  Organ __norm_key__
0    Breast cancer  Breast and female reproductive system         BRCA
1        Carcinoid                               Pancreas          NEC
2  Cervical cancer  Breast and female reproductive system         CESC
📄 Other table head with __norm_key__:
       cancer                        infiltrates   z_score         p  \
0  ACC (n=79)   T cell CD8+ central memory_XCELL -0.358919  0.719656   
1  ACC (n=79)  T cell CD8+ effector memory_XCELL  0.832973  0.404860   
2  ACC (n=79)            T cell CD8+ naive_XCELL -1.110490  0.266788   

      adj.p __norm_key__  
0  0.935318          ACC  
1  0.714360          ACC  
2  0.570439          ACC  

✅ Test complete. Normalized DataFrames:
               Cancer __norm_key__
0       Breast cancer         BRCA
1           Carcinoid          NEC
2     Cervical cancer         CESC
3   Colorectal cancer         COAD
4  Endometrial cancer     

In [10]:
result

,Cancer,Organ,__norm_key__,cancer,infiltrates,z_score,p,adj.p
0,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),T cell CD8+ central memory_XCELL,-2.378386,0.017389,0.133799
1,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),T cell CD8+ effector memory_XCELL,-1.226429,0.220037,0.516440
2,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),T cell CD8+ naive_XCELL,-2.076707,0.037829,0.215623
3,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),T cell CD8+_CIBERSORT,-2.918807,0.003514,0.057412
4,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),T cell CD8+_CIBERSORT-ABS,-2.031435,0.042211,0.229222
...,...,...,...,...,...,...,...,...
265,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell CD8+_EPIC,0.969835,0.332129,0.662597
266,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell CD8+_MCPCOUNTER,0.031517,0.974857,0.999741
267,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell CD8+_QUANTISEQ,-0.243034,0.807979,0.971035
268,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell CD8+_TIMER,-0.374166,0.708281,0.935318


In [11]:
master_path="./data_folder/cancer_cancers_master.csv"
folder="data_folder"
master_df = pd.read_csv(master_path)
master_filename = os.path.basename(master_path)

for other_file in os.listdir(folder):
    if not other_file.endswith(".csv") or other_file == master_filename:
        continue

In [12]:
client = OpenAI()
merged_df = merge_tables_semantically(
    master_path="./data_folder/cancer_cancers_master.csv",
    folder="data_folder",
    client=client
)

merged_df.to_csv("merged_output.csv", index=False)
c=pd.read_csv("merged_output.csv")
c


📄 Processing gene_table.csv
🧪 Master table head with __norm_key__:
            Cancer                                  Organ __norm_key__
0    Breast cancer  Breast and female reproductive system         BRCA
1        Carcinoid                               Pancreas          NEC
2  Cervical cancer  Breast and female reproductive system         CESC
📄 Other table head with __norm_key__:
       cancer                  infiltrates       rho         p     adj.p  \
0  ACC (n=79)      Macrophage M0_CIBERSORT  0.156306  0.186646  0.311760   
1  ACC (n=79)  Macrophage M0_CIBERSORT-ABS  0.164890  0.163297  0.289128   
2  ACC (n=79)      Macrophage M1_CIBERSORT  0.211400  0.072593  0.158082   

  __norm_key__  
0          ACC  
1          ACC  
2          ACC  
✅ Merge done. Preview:
          Cancer                                  Organ __norm_key__  \
0  Breast cancer  Breast and female reproductive system         BRCA   
1  Breast cancer  Breast and female reproductive system         BRCA  

,Cancer,Organ,__norm_key__,cancer,infiltrates,rho,p,adj.p,cancer_outcome_table,infiltrates_outcome_table,z_score,p_outcome_table,adj.p_outcome_table
0,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),Macrophage M0_CIBERSORT,-0.114518,0.000297,0.001476,BRCA (n=1100),T cell CD8+ central memory_XCELL,-2.378386,0.017389,0.133799
1,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),Macrophage M0_CIBERSORT,-0.114518,0.000297,0.001476,BRCA (n=1100),T cell CD8+ effector memory_XCELL,-1.226429,0.220037,0.516440
2,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),Macrophage M0_CIBERSORT,-0.114518,0.000297,0.001476,BRCA (n=1100),T cell CD8+ naive_XCELL,-2.076707,0.037829,0.215623
3,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),Macrophage M0_CIBERSORT,-0.114518,0.000297,0.001476,BRCA (n=1100),T cell CD8+_CIBERSORT,-2.918807,0.003514,0.057412
4,Breast cancer,Breast and female reproductive system,BRCA,BRCA (n=1100),Macrophage M0_CIBERSORT,-0.114518,0.000297,0.001476,BRCA (n=1100),T cell CD8+_CIBERSORT-ABS,-2.031435,0.042211,0.229222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12348,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell regulatory (Tregs)_XCELL,-0.046216,0.308269,0.455806,THCA (n=509),T cell CD8+_MCPCOUNTER,0.031517,0.974857,0.999741
12349,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell regulatory (Tregs)_XCELL,-0.046216,0.308269,0.455806,THCA (n=509),T cell CD8+_QUANTISEQ,-0.243034,0.807979,0.971035
12350,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell regulatory (Tregs)_XCELL,-0.046216,0.308269,0.455806,THCA (n=509),T cell CD8+_TIMER,-0.374166,0.708281,0.935318
12351,Thyroid cancer,Endocrine tissues,THCA,THCA (n=509),T cell regulatory (Tregs)_XCELL,-0.046216,0.308269,0.455806,THCA (n=509),T cell CD8+_XCELL,-0.388731,0.697475,0.933589
